# producing database
# read files with no rows dropped, concat together 

In [1]:
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from pandas.io.parsers import read_csv


In [2]:
sbg=pd.read_excel('half-way/'+'SBG'+'.xlsx')
sbg=sbg.loc[:,['year','PE','tech_type', 'Bezirk','Gemeinde','KG_NR','KG']]
sbg['BL']='Salzburg'
sbg.groupby('tech_type').PE.sum()

tech_type
BKF          4484.333333
Bel.         6904.266667
Fest         1080.000000
Kompost       124.133333
MBR           268.000000
PKA          5490.066667
Primary      4406.666667
RBC           598.333333
SBR          4640.466667
Tropf        1692.693333
Unbekannt     912.666667
Name: PE, dtype: float64

In [3]:
karn=pd.read_excel('half-way/'+'karn'+'.xlsx')
karn=karn.loc[:,['year','PE','tech_type', 'Bezirk','Gemeinde','KG_NR','KG']]
karn['BL']='Carinthia'
karn.groupby('tech_type').PE.sum()


tech_type
BKF           2928.133333
Bel.         40017.213333
Fest           476.666667
PKA           5451.786667
Primary      40933.733333
RBC             84.666667
SBR           5762.733333
Tropf          178.000000
Unbekannt     5374.293151
Name: PE, dtype: float64

In [4]:
noe=pd.read_excel('half-way/'+'noe'+'.xlsx')
noe=noe[['INBETRIEBNAHME','EW60','tech_type','BEZIRK','NAME']]
noe.rename(columns={'INBETRIEBNAHME':'year','EW60':'PE','BEZIRK':'Bezirk' },inplace=True)
noe['BL']='Lower Austria'
noe[noe.tech_type=='Unbekannt'].PE.sum()
noe.groupby('tech_type').PE.sum()

tech_type
Andere        2070.0
BKF           2624.0
Bel.         20689.0
Fest           324.0
Kompost          5.0
MBR             52.0
PKA          13263.0
Primary       4372.0
RBC            203.0
SBR          63123.0
Tropf         1633.0
Unbekannt    14832.0
Wirbel          18.0
Name: PE, dtype: float64

In [5]:
tirol=pd.read_excel('half-way/'+'tirol'+'.xlsx')
tirol=tirol[['year','PE','tech_type', 'Bezirk','Gemeinde','KG_NR','KG']]
tirol['BL']='Tyrol'
tirol.groupby('tech_type').PE.sum()

tech_type
0            3372.666667
BKF          1029.500000
Bel.         7959.533333
Fest          213.000000
Kompost       461.333333
PKA          1857.966667
Primary     10321.200000
Rotation      444.000000
SBR          3933.026667
Tropf        2403.000000
Wirbel        390.000000
Name: PE, dtype: float64

In [6]:
steyr=pd.read_excel('half-way/'+'steyr'+'.xlsx')
steyr=steyr[['year','PE','tech_type', 'Gemeinde','KG_NR','KG','Name']]
steyr['BL']='Stirya'
steyr.groupby('tech_type').PE.sum()

tech_type
0               49
Andere         125
BKF           3353
Bel.         24076
Fest          3031
MBR             10
PKA          26356
Primary      18683
SBR          10723
Tauch          367
Tropf         3653
Unbekannt      132
Name: PE, dtype: int64

In [7]:
oebo=pd.read_excel('half-way/'+'oebo'+'.xlsx')
oebo=oebo[['INBETRIEBNAHME','EW60','Verfahren', 'ANLAGENAME']]
oebo.rename(columns={'INBETRIEBNAHME':'year','EW60':'PE','Verfahren':'tech_type'},inplace=True)
oebo['BL']='Upper Austria'
oebo.groupby('tech_type').PE.sum()

tech_type
Belebung      12593
Belebung +     1344
Filtration      180
PF             8799
SBR           15166
SBR +          5703
Sonstig        2237
mechanisch     2150
Name: PE, dtype: int64

In [8]:
#load minor states
BRG=pd.read_excel('half-way/complete/BRG.xlsx')
VRB=pd.read_excel('half-way/complete/VBG.xlsx')
VIE=pd.read_excel('half-way/complete/wien.xlsx')

In [9]:
files=[karn,steyr, noe,tirol,oebo,sbg, VIE,VRB,BRG]

In [10]:
data=pd.concat(files)

### remove mistkaes, unify categories


In [11]:
data.replace('Bewilligungzeitpunkt abgelaufen oder unbekannt',0,inplace=True)
data.replace('Bescheid mangelhaft',0,inplace=True)
data.replace('m³ pro Tag x 1000 / 150',0,inplace=True)
data.replace('[Liter pro Sekunde x 3600 (= 1 Std.) x 10] / 150',0,inplace=True)
data=data.fillna(0).astype({'PE':int})



In [12]:
data.tech_type=data.tech_type.replace(0, 'Unbekannt')
data.tech_type=data.tech_type.replace('Belebung', 'Bel.')
data.tech_type=data.tech_type.replace('Belebung +', 'Bel.')
data.tech_type=data.tech_type.replace('PF', 'PKA')
data.tech_type=data.tech_type.replace('Rotation', 'RBC')
data.tech_type=data.tech_type.replace('SBR +', 'SBR')
data.tech_type=data.tech_type.replace('Sonstig', 'Other')
data.tech_type=data.tech_type.replace('mechanisch', 'Primary')
data.tech_type=data.tech_type.replace('3-k', 'Primary')
data.tech_type=data.tech_type.replace('Filtersack', 'Primary')
data.tech_type=data.tech_type.replace('Kompost', 'Other')
data.tech_type=data.tech_type.replace('Filterkammer', 'Primary')



In [13]:
rep={'Kompost':'Other','Wirbel':'Other','0':'Unknown','Unbekannt':'Unknown','Andere':'Other'}

In [14]:
data.tech_type=data.tech_type.replace(rep)


In [15]:
data.groupby('tech_type').PE.sum()

tech_type
BKF            15236
Bel.          113969
Fest            5144
Filtration       180
MBR              330
Other           5428
PKA            61946
Primary        81530
RBC             1479
SBR           111272
Tauch            367
Tropf           9676
Unknown        26164
Name: PE, dtype: int32

In [16]:
data[data.BL=='Lower-Austria'].groupby('tech_type').PE.sum()

Series([], Name: PE, dtype: int32)

In [17]:
data=data.astype({'year':int})

# BEzirk, Gemeinde and KG_NR need to be cleaned and unified. either number or string

In [18]:
data.to_excel('final/database.xlsx',index=False)

In [19]:
import pandas as pd
test=pd.read_excel('final/database.xlsx')
test.groupby('BL').count()

,year,PE,tech_type,Bezirk,Gemeinde,KG_NR,KG,Name,NAME,ANLAGENAME,Adresse,ID Nr.
BL,,,,,,,,,,,,
Burgenland,23,23,23,23,23,23,23,23,23,23,23,23
Carinthia,7276,7276,7276,7276,7276,7276,7276,7276,7276,7276,7276,7276
Lower Austria,4835,4835,4835,4835,4835,4835,4835,4835,4835,4835,4835,4835
Salzburg,1732,1732,1732,1732,1732,1732,1732,1732,1732,1732,1732,1732
Stirya,10718,10718,10718,10718,10718,10718,10718,10718,10718,10718,10718,10718
Tyrol,1294,1294,1294,1294,1294,1294,1294,1294,1294,1294,1294,1294
Upper Austria,2633,2633,2633,2633,2633,2633,2633,2633,2633,2633,2633,2633
Vienna,25,25,25,25,25,25,25,25,25,25,25,25
Vorarlberg,156,156,156,156,156,156,156,156,156,156,156,156


In [20]:
test.groupby('BL').count()

,year,PE,tech_type,Bezirk,Gemeinde,KG_NR,KG,Name,NAME,ANLAGENAME,Adresse,ID Nr.
BL,,,,,,,,,,,,
Burgenland,23,23,23,23,23,23,23,23,23,23,23,23
Carinthia,7276,7276,7276,7276,7276,7276,7276,7276,7276,7276,7276,7276
Lower Austria,4835,4835,4835,4835,4835,4835,4835,4835,4835,4835,4835,4835
Salzburg,1732,1732,1732,1732,1732,1732,1732,1732,1732,1732,1732,1732
Stirya,10718,10718,10718,10718,10718,10718,10718,10718,10718,10718,10718,10718
Tyrol,1294,1294,1294,1294,1294,1294,1294,1294,1294,1294,1294,1294
Upper Austria,2633,2633,2633,2633,2633,2633,2633,2633,2633,2633,2633,2633
Vienna,25,25,25,25,25,25,25,25,25,25,25,25
Vorarlberg,156,156,156,156,156,156,156,156,156,156,156,156


In [21]:
test.groupby('tech_type').count()

,year,PE,Bezirk,Gemeinde,KG_NR,KG,BL,Name,NAME,ANLAGENAME,Adresse,ID Nr.
tech_type,,,,,,,,,,,,
BKF,1262,1262,1262,1262,1262,1262,1262,1262,1262,1262,1262,1262
Bel.,7317,7317,7317,7317,7317,7317,7317,7317,7317,7317,7317,7317
Fest,475,475,475,475,475,475,475,475,475,475,475,475
Filtration,24,24,24,24,24,24,24,24,24,24,24,24
MBR,27,27,27,27,27,27,27,27,27,27,27,27
Other,241,241,241,241,241,241,241,241,241,241,241,241
PKA,5718,5718,5718,5718,5718,5718,5718,5718,5718,5718,5718,5718
Primary,6432,6432,6432,6432,6432,6432,6432,6432,6432,6432,6432,6432
RBC,65,65,65,65,65,65,65,65,65,65,65,65
